In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import pandas as pd

In [2]:
# Initialize feature and target datasets
X = pd.read_csv(r"C:\Programming\Python\Industry Level Projects\Django E-t-E Project\myproject\notebooks\features.csv")
y = pd.read_csv(r"C:\Programming\Python\Industry Level Projects\Django E-t-E Project\myproject\notebooks\target.csv")

X.head()

,age,marital,credit_on_default,annual_balance,housing_loan,personal_loan,contact_day,contact_duration,campaign_contacts,days_last_contacted,...,contact_month_dec,contact_month_feb,contact_month_jan,contact_month_jul,contact_month_jun,contact_month_mar,contact_month_may,contact_month_nov,contact_month_oct,contact_month_sep
0,59,1,0,2343,1,0,5,1042,1,-1,...,False,False,False,False,False,False,True,False,False,False
1,56,1,0,45,0,0,5,1467,1,-1,...,False,False,False,False,False,False,True,False,False,False
2,41,1,0,1270,1,0,5,1389,1,-1,...,False,False,False,False,False,False,True,False,False,False
3,55,1,0,2476,1,0,5,579,1,-1,...,False,False,False,False,False,False,True,False,False,False
4,54,1,0,184,0,0,5,673,2,-1,...,False,False,False,False,False,False,True,False,False,False


In [12]:
# Split data into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=42)

# Train demo model to get feature importances
demo_model = RandomForestClassifier(random_state=42)
demo_model.fit(X_train, y_train)
feature_importances = demo_model.feature_importances_

# Map importances to their features
feature_importance_df = pd.DataFrame({
    "Feature" : X.columns,
    'Importance' : feature_importances
})

feature_importance_df.head()

C:\Users\LESEGO\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,Feature,Importance
0,age,0.078876
1,marital,0.013463
2,credit_on_default,0.001496
3,annual_balance,0.080912
4,housing_loan,0.025059


In [23]:
feature_importance_df.sort_values(by='Importance', ascending=False, inplace=True)  # Sort features by most important

selected_features = list(feature_importance_df['Feature'][0:5])  # Put selected features into list

feature_importance_df.head()

,Feature,Importance
7,contact_duration,0.352638
3,annual_balance,0.080912
0,age,0.078876
6,contact_day,0.070257
11,poutcome,0.044352


In [32]:
# Filter out irrelevant features
X_train = X_train[selected_features]
X_test = X_test[selected_features]

X_train.head()

,contact_duration,annual_balance,age,contact_day,poutcome
5304,98,8121,60,18,0
7512,281,-55,51,11,0
4700,712,-1042,28,6,0
4918,566,4457,30,21,0
4450,252,2544,31,15,1


In [33]:
# Scale X data for better model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Assert that the scaled data has the same shape as the original values
assert(X_train.shape == X_train_scaled.shape)
assert(X_test.shape == X_test_scaled.shape)

In [39]:
param_grid = {
    'n_estimators': [100, 200, 300],          # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],          # Depth of each tree
    'min_samples_split': [2, 5, 10],          # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],            # Minimum samples required to be at a leaf node
    'max_features': ['sqrt', 'log2'],         # Number of features to consider for best split
    'bootstrap': [True, False],               # Whether to use bootstrap samples
}

rf_model = RandomForestClassifier(random_state=42)  # Intialize model

grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv = 2,
    n_jobs=-1,
    verbose=2
)
grid_search.fit(X_train_scaled, y_train)  # Fit the model

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_}")

# Save the model
best_model = grid_search.best_estimator_
joblib.load(best_model, r"myapp/ml_models/rf_model.pkl")

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


KeyboardInterrupt: 